# Code template for hand-in on reconstruction attacks.

Advanced Topics in Machine Learning, U. Copenhagen, fall 2024

Created by Rasmus Pagh with minor edits by Thomas Christensen

Queries on a hidden dataset x from {-1,+1}^100 can be made via the query method below
which calls a web API that allows dot product queries with vectors in {-1,+1}^100.
To protect data, Laplace noise is added to responses. Using the techniques you have
seen in the lecture it is possible to partially reconstruct the dataset using 200 queries.
To make sure that you get a unique challenge, choose any unique string as your challenge
identifier. The web API will keep track of the number of queries made for each identifier.

# Support function for querying the web API

In [66]:
import numpy as np
import requests as rq
import sys
np.set_printoptions(threshold=sys.maxsize)
# Retrieve answer to challenge for a given query
def query(challenge_id, query_vector, submit=False):
    # Only alphanumeric challenge_id and vextor entries in {-1,+1} are allowed:
    assert(challenge_id.isalnum())
    assert(np.max(np.minimum(np.abs(query_vector-1),np.abs(query_vector+1)))==0)

    # if query array is 1d, make it 2d
    if query_vector.ndim == 1:
        query_vector = query_vector.reshape(1,-1)

    payload = { 'challengeid': challenge_id, 'submit': submit,
                'query': str(query_vector.tolist()) }
    response = rq.post("https://baconbreaker.pythonanywhere.com/query", data = payload).json()
    if submit == False:
        return np.array(eval(response['result']))
    else:
        return response['result']

# Making random queries to the API

In [100]:
challenge_id = 'belongfossildecrease' # identifier for hidden dataset
n = 100 # number of entries in hidden dataset
num_queries = 2*n # number of queries to be asked

queries = np.random.choice([-1,+1], size=(num_queries,n)) # Set of random queries
query_results = query(challenge_id, queries)

# Check the results
if query_results is None or len(query_results) != num_queries:
    raise ValueError("Query results are invalid or incomplete.")

print(query_results)

[  4. -26.  -9. -19.  -9.  15.  -3.  17.  -8. -12.   5.  -7. -17.   3.
 -13.   2. -14.  -8.  -6.  -3.  -0. -20.   3.   2.  10. -14.  20.  13.
   1.  -2. -17.  -4.  14.   6.  15.   2.  32.  16.  21.   2. -29.   2.
 -26.   4.   4. -25.  20.   1. -13.  18.   3. -42.  21. -21.  11.   6.
 -19.  16.  22. -19.  -1.   1.  -6.   8. -36.  -7. -10.   6.  10.  -3.
  -3.   9.  16.  11.   1.  -2.  -3.   6. -16.   2.  10.   2.  -2.  13.
 -13.  56. -11.  -1.  -2.  13. -12.   3. -15.   7. -28.  12.  19.   2.
  -8. -19.  25. -22.  14.  -8.  -5.  12.   2.  -8.   3.   2.  31. -25.
 -15.  13.   2.  18.  -5. -14.   5. -16.  -2.   1.  15.   1.   7. -30.
  -4.  12.  18.  14.   1.  57. -11.   4. -11.  -4.   7. -43.  -7.  15.
 -19.  20.   8.  22. -25.  -7.  -8. -28. -12.   0. -15.  -0.  -8.  -4.
 -21. -41.  -6. -85.  11. -24.  10.  -8.   8.   7.  -6. -14.  39. -23.
 -11.   7.  25.  14. -14.  -5.  -9.  17.  22.   6. -28.  28.   9.   6.
  -9.   1. -10.   8.  19.   4.  -4.  -8.  11.  -4.  -9. -42. -18.  -8.
   6. 

# Baseline reconstruction attack: Guess equals the best query

Copy the notebook and replace with your attack. The first submission for a given challenge_id is stored on the server and on the leaderboard: http://baconbreaker.pythonanywhere.com/leaderboard/?n=100

**Tip**: to solve a linear program you can use the following import:

```
from scipy.optimize import linprog
```



In [106]:
from scipy.optimize import linprog

# Decision variables: x (the dataset) and e (absolute errors)
c = np.concatenate([np.zeros(n), np.ones(num_queries)])

# Constraints:
A_ub = np.block([
    [queries, -np.eye(num_queries)], # Handles upper bound constraint
    [-queries, -np.eye(num_queries)],  # Handles lower bound constraint
    [np.eye(n), np.zeros((n, num_queries))], # Enforces -1 <= x_j
    [-np.eye(n), np.zeros((n, num_queries))] # Enforces x_j <= 1
])

b_ub = np.concatenate([
    query_results,
    -query_results,
    np.ones(n),
    np.ones(n)
])

# Bounds for x in [-1, 1]
bounds = [(-1, 1)] * n + [(0, None)] * num_queries

# Solving the linear program
result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds)

# Extracting the reconstructed dataset
x_reconstructed = np.sign(result.x[:n])

# Submitting the result
submission_result = query(challenge_id, x_reconstructed, submit=True)
# Printing the submission result
print(f"Reconstruction attack achieves fraction {(1 + submission_result / n) / 2} correct values")

Reconstruction attack achieves fraction 0.8300000000000001 correct values
